<h1 align=center><font size = 5>Data Science Capstone Final Project: Start a new restaurant in Madison, WI</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Introduction</a>
2. <a href="#item2">Data </a>
3. <a href="#item3">Methodology</a>
4. <a href="#item4">Analysis</a>
5. <a href="#item5">Results and Dicussion</a>   
6. <a href="#item5">Conclusion</a>   
</font>
</div>

## Introduction
Madison, the capital city of Wisconsin, lies west of Milwaukee. In 2018, Madison's estimated population is around 258,054, it rank on top list of the popular cities in US. It is located between Lake Mendota and Lake Monona, and is the home to the University of Wisconsin–Madison. Madison is a growing technology economy and the region is home to the headquarters of Epic Systems, American Family Insurance, Exact Sciences, Promega, American Girl, Sub-Zero, Lands' End, a regional office for Google, the University Research Park, as well as many biotech and health systems startups. So many education and science institutes and large companies in Madison means a hurge demand of food, thus it is a good business opportunity to start a restaurant here. Before investing a new restaurant, we should define concept of the restaurant and find the optimal location for it. This project used the current restaurants information to figure out what kind of restaurants to start and which locations is the good choice for a restaurant investor.


## Data
Inorder to do restaurants analysis the data will be collected from Foursquare location data.
**Restaurants Data**
The surrounding restaurants data can obtained from Foursquare API by using request function by passing url which is the path to access the data we need. The url will be like 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format( CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)". After cleaning the data set, we will get the clean data containing the restaurant name, restaurants category, its geographical coordinate and its rating score.

### Import restaurants data

First, import libraries and packeage we need in this project.

In [1]:
! pip install lxml
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

! pip install geocoder
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [5]:
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


Let's get the Maidson, WI geographical coordinate from Foursquare

In [6]:
g = geocoder.arcgis('Madison, Wisconsin')
lat_lng_coords = g.latlng
lat_lng_coords

[43.07313000000005, -89.38643999999994]

In [7]:
# The code was removed by Watson Studio for sharing.
# @hidden_cell
CLIENT_ID = 'SVZB02PCX2UEYMDWDRHTLWG5M4JALVE1V3MZ3OJCNNXO35OG'
CLIENT_SECRET = 'ZWQGTHPNUFHQDIL4QJZP2K5NJZNO4FIHMI2XQB0ZQT2EMTVO'

Get the basic restaurants data 

In [9]:
radius=10000
LIMIT=100
VERSION = '20200309'
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
search_query = 'restaurant'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b5236c6f964a520447027e3,226 State St,...,[226 State St (btwn W. Johnson & W. Dayton St....,"[{'label': 'display', 'lat': 43.07485603062506...",43.074856,-89.389314,NaN,53703,WI,Nick's Restaurant & Bar,v-1584912519,NaN
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b49211af964a5201a6726e3,108 King St,...,"[108 King St, Madison, WI 53703, United States]","[{'label': 'display', 'lat': 43.07453928292850...",43.074539,-89.379341,Downtown Madison,53703,WI,Restaurant Muramoto,v-1584912519,NaN
2,"[{'id': '5454152e498ef71e2b9132c6', 'name': 'E...",324085,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/crandalls-p...,False,51fc3d8d498e686ce9e667e6,334 State Street,...,"[334 State Street (Gorham Street), Madison, WI...","[{'label': 'display', 'lat': 43.074973, 'lng':...",43.074973,-89.390995,NaN,53703,WI,Crandall's Peruvian Restaurant & Catering,v-1584912519,199285184
3,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,4afcc582f964a520bc2522e3,23 N Pinckney St,...,"[23 N Pinckney St, Madison, WI 53703, United S...","[{'label': 'display', 'lat': 43.076153, 'lng':...",43.076153,-89.383526,Downtown Madison,53703,WI,The Old Fashioned Tavern & Restaurant,v-1584912519,196015844
4,"[{'id': '4deefc054765f83613cdba6f', 'name': 'I...",262846,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/bandung-ind...,False,4b634d74f964a520c9702ae3,600 Williamson St,...,"[600 Williamson St (btw Blair & Blount), Madis...","[{'label': 'display', 'lat': 43.07678768970482...",43.076788,-89.374243,NaN,53703,WI,Bandung Indonesian Restaurant,v-1584912519,464606589


Modify the data fame 

In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
mad_restaurant = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
mad_restaurant['categories'] = mad_restaurant.apply(get_category_type, axis=1)

# clean column names by keeping only last term
mad_restaurant.columns = [column.split('.')[-1] for column in mad_restaurant.columns]

mad_restaurant

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Nick's Restaurant & Bar,American Restaurant,226 State St,US,Madison,United States,btwn W. Johnson & W. Dayton St.,302,[226 State St (btwn W. Johnson & W. Dayton St....,"[{'label': 'display', 'lat': 43.07485603062506...",43.074856,-89.389314,NaN,53703,WI,4b5236c6f964a520447027e3
1,Restaurant Muramoto,Sushi Restaurant,108 King St,US,Madison,United States,NaN,598,"[108 King St, Madison, WI 53703, United States]","[{'label': 'display', 'lat': 43.07453928292850...",43.074539,-89.379341,Downtown Madison,53703,WI,4b49211af964a5201a6726e3
2,Crandall's Peruvian Restaurant & Catering,Event Service,334 State Street,US,Madison,United States,Gorham Street,423,"[334 State Street (Gorham Street), Madison, WI...","[{'label': 'display', 'lat': 43.074973, 'lng':...",43.074973,-89.390995,NaN,53703,WI,51fc3d8d498e686ce9e667e6
3,The Old Fashioned Tavern & Restaurant,Gastropub,23 N Pinckney St,US,Madison,United States,NaN,411,"[23 N Pinckney St, Madison, WI 53703, United S...","[{'label': 'display', 'lat': 43.076153, 'lng':...",43.076153,-89.383526,Downtown Madison,53703,WI,4afcc582f964a520bc2522e3
4,Bandung Indonesian Restaurant,Indonesian Restaurant,600 Williamson St,US,Madison,United States,btw Blair & Blount,1072,"[600 Williamson St (btw Blair & Blount), Madis...","[{'label': 'display', 'lat': 43.07678768970482...",43.076788,-89.374243,NaN,53703,WI,4b634d74f964a520c9702ae3
5,Wah Kee Noodle Restaurant,Asian Restaurant,600 Williamson St,US,Madison,United States,NaN,1006,"[600 Williamson St, Madison, WI 53703, United ...","[{'label': 'display', 'lat': 43.07654305371098...",43.076543,-89.374983,Marquette,53703,WI,4b85c40af964a520c17131e3
6,Coliseum Bar and Restaurant,American Restaurant,232 E Olin Ave,US,Madison,United States,John Nolen,2419,"[232 E Olin Ave (John Nolen), Madison, WI 5371...","[{'label': 'display', 'lat': 43.05201124423912...",43.052011,-89.379409,NaN,53713,WI,41326e00f964a520c9151fe3
7,Maharani Indian Restaurant,Indian Restaurant,380 W Washington Ave,US,Madison,United States,N Broom St,268,"[380 W Washington Ave (N Broom St), Madison, W...","[{'label': 'display', 'lat': 43.07150966807177...",43.071510,-89.388882,NaN,53703,WI,4b803cb7f964a520925f30e3
8,Peacemeal Vegetarian Restaurant,Vegetarian / Vegan Restaurant,115 State St,US,Madison,United States,NaN,166,"[115 State St, Madison, WI 53703, United States]","[{'label': 'display', 'lat': 43.0745, 'lng': -...",43.074500,-89.387248,NaN,53703,WI,4f43672719834bc91f569240
9,Orpheum Theatre Lobby Restaurant,Café,216 State St,US,Madison,United States,NaN,275,"[216 State St, Madison, WI 53703, United States]","[{'label': 'display', 'lat': 43.07497, 'lng': ...",43.074970,-89.388707,NaN,53703,WI,4e4e53fabd4101d0d7a7d224


In [11]:
#mad_restaurant.name
mad_restaurant.shape

(50, 16)

### Visualize the rsetaurants in Map. 
The map will show us the distribution of these restaurants.

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent downtown Madison
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Maidson',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(mad_restaurant.lat, mad_restaurant.lng, mad_restaurant.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Get the ratings of the restaurants

The rating score will be combined into restaurant data. Rating is the a very improtant information of a restaurant, so we can learn better about the restaurant, and use the information to explore furture. Also, we can using rating to be a criteria to cluster the restaurants around Madison. 

In [13]:
rating = []
for venue_id in mad_restaurant.id:    
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #print(result['response']['venue'].keys())
    venue = result['response']['venue']
    if 'rating' in venue.keys():
        #print(venue['rating'])
        rating.append(venue['rating'])
    else:
        rating.append(None)
rating

[6.8,
 8.5,
 6.8,
 9.0,
 8.7,
 7.4,
 6.7,
 6.9,
 None,
 None,
 7.5,
 None,
 None,
 None,
 None,
 6.2,
 8.0,
 None,
 7.0,
 6.4,
 6.0,
 5.6,
 None,
 None,
 7.6,
 7.4,
 5.9,
 7.4,
 7.8,
 7.7,
 6.5,
 None,
 None,
 None,
 7.1,
 None,
 8.3,
 None,
 None,
 None,
 None,
 8.1,
 6.2,
 5.4,
 None,
 7.9,
 8.6,
 7.2,
 None,
 None]

In [14]:
mad_restaurant['rating'] = rating
#drop NA in the rating column 
mad_restaurant.dropna(subset=['rating'], how = 'all', inplace = True)
mad_restaurant.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,rating
0,Nick's Restaurant & Bar,American Restaurant,226 State St,US,Madison,United States,btwn W. Johnson & W. Dayton St.,302,[226 State St (btwn W. Johnson & W. Dayton St....,"[{'label': 'display', 'lat': 43.07485603062506...",43.074856,-89.389314,NaN,53703,WI,4b5236c6f964a520447027e3,6.8
1,Restaurant Muramoto,Sushi Restaurant,108 King St,US,Madison,United States,NaN,598,"[108 King St, Madison, WI 53703, United States]","[{'label': 'display', 'lat': 43.07453928292850...",43.074539,-89.379341,Downtown Madison,53703,WI,4b49211af964a5201a6726e3,8.5
2,Crandall's Peruvian Restaurant & Catering,Event Service,334 State Street,US,Madison,United States,Gorham Street,423,"[334 State Street (Gorham Street), Madison, WI...","[{'label': 'display', 'lat': 43.074973, 'lng':...",43.074973,-89.390995,NaN,53703,WI,51fc3d8d498e686ce9e667e6,6.8
3,The Old Fashioned Tavern & Restaurant,Gastropub,23 N Pinckney St,US,Madison,United States,NaN,411,"[23 N Pinckney St, Madison, WI 53703, United S...","[{'label': 'display', 'lat': 43.076153, 'lng':...",43.076153,-89.383526,Downtown Madison,53703,WI,4afcc582f964a520bc2522e3,9.0
4,Bandung Indonesian Restaurant,Indonesian Restaurant,600 Williamson St,US,Madison,United States,btw Blair & Blount,1072,"[600 Williamson St (btw Blair & Blount), Madis...","[{'label': 'display', 'lat': 43.07678768970482...",43.076788,-89.374243,NaN,53703,WI,4b634d74f964a520c9702ae3,8.7
5,Wah Kee Noodle Restaurant,Asian Restaurant,600 Williamson St,US,Madison,United States,NaN,1006,"[600 Williamson St, Madison, WI 53703, United ...","[{'label': 'display', 'lat': 43.07654305371098...",43.076543,-89.374983,Marquette,53703,WI,4b85c40af964a520c17131e3,7.4
6,Coliseum Bar and Restaurant,American Restaurant,232 E Olin Ave,US,Madison,United States,John Nolen,2419,"[232 E Olin Ave (John Nolen), Madison, WI 5371...","[{'label': 'display', 'lat': 43.05201124423912...",43.052011,-89.379409,NaN,53713,WI,41326e00f964a520c9151fe3,6.7
7,Maharani Indian Restaurant,Indian Restaurant,380 W Washington Ave,US,Madison,United States,N Broom St,268,"[380 W Washington Ave (N Broom St), Madison, W...","[{'label': 'display', 'lat': 43.07150966807177...",43.071510,-89.388882,NaN,53703,WI,4b803cb7f964a520925f30e3,6.9
10,The Statehouse Restaurant,American Restaurant,1001 Wisconsin Pl,US,Madison,United States,The Edgewater Hotel,717,"[1001 Wisconsin Pl (The Edgewater Hotel), Madi...","[{'label': 'display', 'lat': 43.07908716343267...",43.079087,-89.389802,NaN,53703,WI,5428c94f498eb5a73262d8a2,7.5
15,Porta Bella Italian Restaurant,Italian Restaurant,425 N Frances St,US,Madison,United States,NaN,739,"[425 N Frances St, Madison, WI 53703, United S...","[{'label': 'display', 'lat': 43.07397001583682...",43.073970,-89.395459,NaN,53703,WI,4b0ae735f964a520c12923e3,6.2


### The final data used to analysis
Simplify the Madison restaurant raw data. To make the restaurant data easier to read, we can select the columns which are most interested to us, like name, categories, geographical coordinate, and ratings. Then, we can sort the data to see the top rating restaurants. 

In [15]:
mad_rest = mad_restaurant[['name', 'categories', 'lat', 'lng', 'rating']]
mad_rest

,name,categories,lat,lng,rating
0,Nick's Restaurant & Bar,American Restaurant,43.074856,-89.389314,6.8
1,Restaurant Muramoto,Sushi Restaurant,43.074539,-89.379341,8.5
2,Crandall's Peruvian Restaurant & Catering,Event Service,43.074973,-89.390995,6.8
3,The Old Fashioned Tavern & Restaurant,Gastropub,43.076153,-89.383526,9.0
4,Bandung Indonesian Restaurant,Indonesian Restaurant,43.076788,-89.374243,8.7
5,Wah Kee Noodle Restaurant,Asian Restaurant,43.076543,-89.374983,7.4
6,Coliseum Bar and Restaurant,American Restaurant,43.052011,-89.379409,6.7
7,Maharani Indian Restaurant,Indian Restaurant,43.071510,-89.388882,6.9
10,The Statehouse Restaurant,American Restaurant,43.079087,-89.389802,7.5
15,Porta Bella Italian Restaurant,Italian Restaurant,43.073970,-89.395459,6.2


### Methodology
In this project, we will using the restaurants information to analysis the popular restaurant type and top choices for the new resetaurant location.

1. To figure out what type of restaurant is suggested to start at the Madison area, we want to know what kind restaurant is more popular in the area, then the rating of each restaurant is ranked and the average of each restaurant category is calculated.

2. To decide where to locate the restaurant, we using K-cluster method to cluster the restaurant by the neary by venues and average the rating of each cluster.


## Analysis

### Sort restaurant 
We sort the restaurants based on the ratings. After sorting, we pick the top 10 rating restaurants.

In [16]:
mad_rest.sort_values(by = 'rating', ascending = False, inplace = True)
mad_rest.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,name,categories,lat,lng,rating
3,The Old Fashioned Tavern & Restaurant,Gastropub,43.076153,-89.383526,9.0
4,Bandung Indonesian Restaurant,Indonesian Restaurant,43.076788,-89.374243,8.7
46,L'etoile,American Restaurant,43.075183,-89.382436,8.6
1,Restaurant Muramoto,Sushi Restaurant,43.074539,-89.379341,8.5
36,Otto's Restaurant & Bar,American Restaurant,43.059997,-89.490601,8.3
41,Jamerica Carribean Restaurant,Caribbean Restaurant,43.084210,-89.363428,8.1
16,Lombardino's Restaurant,Italian Restaurant,43.072215,-89.428862,8.0
45,Essen Haus,German Restaurant,43.076218,-89.376537,7.9
28,Lilliana's Restaurant,Cajun / Creole Restaurant,43.015369,-89.430096,7.8
29,Nitty Gritty Madison,Bar,43.071960,-89.395554,7.7


### Average rating of each restaurants category
Average the ratings over the categories. To decide what kind of restaurants we want to start, it's interested to look at the average ratings of each restaurant catogary and see what kind of restaurant will be the most pupular one at Madison area. 

In [17]:
mad_rest_catGrouped = mad_rest.groupby('categories').mean()
mad_rest_catGrouped.sort_values(by = 'rating', ascending = False)

,lat,lng,rating
categories,,,
Gastropub,43.076153,-89.383526,9.000000
Indonesian Restaurant,43.076788,-89.374243,8.700000
Sushi Restaurant,43.074539,-89.379341,8.500000
Caribbean Restaurant,43.084210,-89.363428,8.100000
German Restaurant,43.076218,-89.376537,7.900000
Cajun / Creole Restaurant,43.015369,-89.430096,7.800000
Greek Restaurant,43.060765,-89.365423,7.500000
American Restaurant,43.062388,-89.405829,7.214286
Mexican Restaurant,43.043956,-89.394384,7.200000


### Nearby venues
Get the nearby venus of each restaurant

In [18]:
## define a function 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Restaurant Name', 
                  'Restaurant Latitude', 
                  'Restaurant Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
mad_venues = getNearbyVenues(names = mad_rest['name'],
                                   latitudes = mad_rest['lat'],
                                   longitudes = mad_rest['lng']
                                  )

The Old Fashioned Tavern & Restaurant
Bandung Indonesian Restaurant
L'etoile
Restaurant Muramoto
Otto's Restaurant & Bar
Jamerica Carribean Restaurant
Lombardino's Restaurant
Essen Haus
Lilliana's Restaurant
Nitty Gritty Madison
Monona Gardens Family Restaurant
The Statehouse Restaurant
Wah Kee Noodle Restaurant
PARTHENON GYROS
Maharana Restaurant
El Pastor Restaurant And Bar
Dairyland Family Restaurant
Parkway Restaurant
Maharani Indian Restaurant
Crandall's Peruvian Restaurant & Catering
Nick's Restaurant & Bar
Coliseum Bar and Restaurant
Himal Chuli
Perkins Restaurant & Bakery
Porta Bella Italian Restaurant
The Curve Restaurant
The City Bar & Restaurant
Edo Japanese Restaurant
Kabul Mediterranean Restaurant
Vientiane Palace


In [20]:
print(mad_venues.shape)
mad_venues.head()

(1492, 7)


,Restaurant Name,Restaurant Latitude,Restaurant Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Old Fashioned Tavern & Restaurant,43.076153,-89.383526,Dane County Farmers' Market,43.075337,-89.384996,Farmers Market
1,The Old Fashioned Tavern & Restaurant,43.076153,-89.383526,Capitol Square,43.074593,-89.384379,Plaza
2,The Old Fashioned Tavern & Restaurant,43.076153,-89.383526,The Old Fashioned Tavern & Restaurant,43.076153,-89.383526,Gastropub
3,The Old Fashioned Tavern & Restaurant,43.076153,-89.383526,Madison Children's Museum,43.076790,-89.384422,Museum
4,The Old Fashioned Tavern & Restaurant,43.076153,-89.383526,Colectivo Coffee,43.074884,-89.381937,Coffee Shop


### Count the venues around the rsetaurants
The number of nearby venues means the surrounding activities, which can give us idea about how much people flow in this area where the restaurant locates.

In [54]:
mad_venues_count= mad_venues.groupby('Restaurant Name').count()
mad_venues_countsort = mad_venues_count.sort_values(by = 'Venue', ascending = False)
mad_venues_countsort.head()

,Restaurant Latitude,Restaurant Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Restaurant Name,,,,,,
Vientiane Palace,100,100,100,100,100,100
Nick's Restaurant & Bar,100,100,100,100,100,100
The Old Fashioned Tavern & Restaurant,100,100,100,100,100,100
Restaurant Muramoto,97,97,97,97,97,97
Crandall's Peruvian Restaurant & Catering,90,90,90,90,90,90


Let form the nearby venues in to a dataframe which can contribute to our final recommendation.

In [55]:
mad_venues_countsort.reset_index(inplace = True)
nearVenues_Num = mad_venues_countsort[['Restaurant Name', 'Restaurant Latitude']]
nearVenues_Num.columns = ['Restaurant Name', 'Nearby Venue counts']
nearVenues_Num

,Restaurant Name,Nearby Venue counts
0,Vientiane Palace,100
1,Nick's Restaurant & Bar,100
2,The Old Fashioned Tavern & Restaurant,100
3,Restaurant Muramoto,97
4,Crandall's Peruvian Restaurant & Catering,90
5,Himal Chuli,87
6,PARTHENON GYROS,87
7,L'etoile,83
8,Kabul Mediterranean Restaurant,75
9,The City Bar & Restaurant,70


### Cluster the restaurant

In [75]:
# one hot encoding
mad_onehot = pd.get_dummies(mad_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mad_onehot['Restaurant Name'] = mad_venues['Restaurant Name'] 

# move neighborhood column to the first column
fixed_columns = [mad_onehot.columns[-1]] + list(mad_onehot.columns[:-1])
mad_onehot = mad_onehot[fixed_columns]

mad_onehot.head()

,Restaurant Name,ATM,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,...,Thrift / Vintage Store,Track,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Old Fashioned Tavern & Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Old Fashioned Tavern & Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Old Fashioned Tavern & Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Old Fashioned Tavern & Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Old Fashioned Tavern & Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
mad_onehot.columns

Index(['Restaurant Name', 'ATM', 'Accessories Store', 'American Restaurant',
       'Arcade', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Assisted Living',
       ...
       'Thrift / Vintage Store', 'Track', 'Trail', 'Video Game Store',
       'Video Store', 'Vietnamese Restaurant', 'Wine Bar', 'Wine Shop',
       'Women's Store', 'Yoga Studio'],
      dtype='object', length=171)

In [77]:
mad_onehot.shape

(1492, 171)

In [78]:
mad_grouped = mad_onehot.groupby('Restaurant Name').mean().reset_index()

In [79]:
## Display the top 10 venues appaering around the restaurants

# define function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## 
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Restaurant Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Restaurant Name'] = mad_grouped['Restaurant Name']

for ind in np.arange(mad_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(mad_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Restaurant Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bandung Indonesian Restaurant,Bar,Yoga Studio,Mexican Restaurant,Asian Restaurant,Coffee Shop,Indonesian Restaurant,Lounge,Rock Club,Restaurant,Record Shop
1,Coliseum Bar and Restaurant,Park,Steakhouse,American Restaurant,Music Venue,Trail,Hotel,Athletics & Sports,Bar,Donut Shop,Dive Bar
2,Crandall's Peruvian Restaurant & Catering,Hotel,Bar,American Restaurant,Coffee Shop,Pub,Sandwich Place,Breakfast Spot,Burger Joint,Middle Eastern Restaurant,Ice Cream Shop
3,Dairyland Family Restaurant,Bar,Bakery,Gym,Diner,Park,Fast Food Restaurant,Pizza Place,Food,Intersection,Cajun / Creole Restaurant
4,Edo Japanese Restaurant,Sandwich Place,Chinese Restaurant,Grocery Store,Donut Shop,Shoe Store,Japanese Restaurant,Mexican Restaurant,Café,Pizza Place,Bar


In [80]:
# set number of clusters
kclusters = 4

mad_grouped_clustering = mad_grouped.drop('Restaurant Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mad_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mad_merged = mad_restaurant

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mad_merged = mad_merged.join(venues_sorted.set_index('Restaurant Name'), on='name')

mad_merged.head() # check the last columns!

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Nick's Restaurant & Bar,American Restaurant,226 State St,US,Madison,United States,btwn W. Johnson & W. Dayton St.,302,[226 State St (btwn W. Johnson & W. Dayton St....,"[{'label': 'display', 'lat': 43.07485603062506...",...,Hotel,Bar,American Restaurant,Coffee Shop,Pub,Sandwich Place,Ice Cream Shop,Steakhouse,Pizza Place,Plaza
1,Restaurant Muramoto,Sushi Restaurant,108 King St,US,Madison,United States,NaN,598,"[108 King St, Madison, WI 53703, United States]","[{'label': 'display', 'lat': 43.07453928292850...",...,Bar,New American Restaurant,Hotel,Sandwich Place,Coffee Shop,American Restaurant,Asian Restaurant,Café,Yoga Studio,Korean Restaurant
2,Crandall's Peruvian Restaurant & Catering,Event Service,334 State Street,US,Madison,United States,Gorham Street,423,"[334 State Street (Gorham Street), Madison, WI...","[{'label': 'display', 'lat': 43.074973, 'lng':...",...,Hotel,Bar,American Restaurant,Coffee Shop,Pub,Sandwich Place,Breakfast Spot,Burger Joint,Middle Eastern Restaurant,Ice Cream Shop
3,The Old Fashioned Tavern & Restaurant,Gastropub,23 N Pinckney St,US,Madison,United States,NaN,411,"[23 N Pinckney St, Madison, WI 53703, United S...","[{'label': 'display', 'lat': 43.076153, 'lng':...",...,Bar,Coffee Shop,American Restaurant,Hotel,New American Restaurant,Pub,Café,Italian Restaurant,Steakhouse,Music Venue
4,Bandung Indonesian Restaurant,Indonesian Restaurant,600 Williamson St,US,Madison,United States,btw Blair & Blount,1072,"[600 Williamson St (btw Blair & Blount), Madis...","[{'label': 'display', 'lat': 43.07678768970482...",...,Bar,Yoga Studio,Mexican Restaurant,Asian Restaurant,Coffee Shop,Indonesian Restaurant,Lounge,Rock Club,Restaurant,Record Shop


In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mad_merged['lat'], mad_merged['lng'], mad_merged['name'], mad_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Average rating of each cluster

In [82]:
mad_merged.columns

Index(['name', 'categories', 'address', 'cc', 'city', 'country', 'crossStreet',
       'distance', 'formattedAddress', 'labeledLatLngs', 'lat', 'lng',
       'neighborhood', 'postalCode', 'state', 'id', 'rating', 'Cluster Labels',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [83]:
mad_merged2 = mad_merged[['name', 'categories','rating', 'Cluster Labels',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']]
mad_merged2.groupby(by = 'Cluster Labels').mean()

,rating
Cluster Labels,
0,8.000000
1,7.126667
2,7.181818
3,6.700000


### Explore the restaurants in each cluster

In [84]:
mad_merged2[mad_merged2['Cluster Labels'] == 0]

,name,categories,rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bandung Indonesian Restaurant,Indonesian Restaurant,8.7,0,Bar,Yoga Studio,Mexican Restaurant,Asian Restaurant,Coffee Shop,Indonesian Restaurant,Lounge,Rock Club,Restaurant,Record Shop
5,Wah Kee Noodle Restaurant,Asian Restaurant,7.4,0,Bar,Yoga Studio,Coffee Shop,Mexican Restaurant,Sandwich Place,Brewery,Rock Club,Record Shop,Bus Stop,Persian Restaurant
45,Essen Haus,German Restaurant,7.9,0,Bar,Coffee Shop,Yoga Studio,American Restaurant,Asian Restaurant,Music Venue,Café,Mexican Restaurant,French Restaurant,Lounge


In [85]:
mad_merged2[mad_merged2['Cluster Labels'] == 1]

,name,categories,rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Nick's Restaurant & Bar,American Restaurant,6.8,1,Hotel,Bar,American Restaurant,Coffee Shop,Pub,Sandwich Place,Ice Cream Shop,Steakhouse,Pizza Place,Plaza
1,Restaurant Muramoto,Sushi Restaurant,8.5,1,Bar,New American Restaurant,Hotel,Sandwich Place,Coffee Shop,American Restaurant,Asian Restaurant,Café,Yoga Studio,Korean Restaurant
2,Crandall's Peruvian Restaurant & Catering,Event Service,6.8,1,Hotel,Bar,American Restaurant,Coffee Shop,Pub,Sandwich Place,Breakfast Spot,Burger Joint,Middle Eastern Restaurant,Ice Cream Shop
3,The Old Fashioned Tavern & Restaurant,Gastropub,9.0,1,Bar,Coffee Shop,American Restaurant,Hotel,New American Restaurant,Pub,Café,Italian Restaurant,Steakhouse,Music Venue
7,Maharani Indian Restaurant,Indian Restaurant,6.9,1,Hotel,Coffee Shop,Pub,Bar,Breakfast Spot,History Museum,Italian Restaurant,Steakhouse,New American Restaurant,Bookstore
10,The Statehouse Restaurant,American Restaurant,7.5,1,American Restaurant,Pub,Hotel,Bar,Hotel Bar,Cocktail Bar,Clothing Store,Miscellaneous Shop,Restaurant,Pizza Place
15,Porta Bella Italian Restaurant,Italian Restaurant,6.2,1,Bar,Salad Place,Sandwich Place,Coffee Shop,Breakfast Spot,Hotel,Burger Joint,Ice Cream Shop,Music Venue,Pizza Place
16,Lombardino's Restaurant,Italian Restaurant,8.0,1,Coffee Shop,Italian Restaurant,Bar,Mobile Phone Shop,Sandwich Place,Bagel Shop,Jewelry Store,Hotel,Convenience Store,Arcade
20,The City Bar & Restaurant,Bar,6.0,1,Bar,Ice Cream Shop,Sandwich Place,Burger Joint,Coffee Shop,Breakfast Spot,Salad Place,Hotel,Pub,Middle Eastern Restaurant
21,Kabul Mediterranean Restaurant,Afghan Restaurant,5.6,1,Bar,Pub,Coffee Shop,Breakfast Spot,Salad Place,Hotel,Sandwich Place,Ice Cream Shop,Burger Joint,Pizza Place


In [86]:
mad_merged2[mad_merged2['Cluster Labels'] == 2]

,name,categories,rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Parkway Restaurant,Diner,7.0,2,Pizza Place,Gas Station,Diner,Motel,Bar,Intersection,Donut Shop,Furniture / Home Store,Sandwich Place,Home Service
19,Perkins Restaurant & Bakery,American Restaurant,6.4,2,Sandwich Place,Furniture / Home Store,Home Service,Plaza,Pizza Place,Intersection,Bar,Motorsports Shop,Fried Chicken Joint,Dance Studio
24,Monona Gardens Family Restaurant,Greek Restaurant,7.6,2,New American Restaurant,Building,American Restaurant,Park,Greek Restaurant,Harbor / Marina,Pet Store,Boat or Ferry,River,Business Service
25,Maharana Restaurant,Indian Restaurant,7.4,2,Furniture / Home Store,Fast Food Restaurant,Coffee Shop,Electronics Store,Gym,Gym / Fitness Center,Sandwich Place,Pet Store,Bookstore,Breakfast Spot
26,Edo Japanese Restaurant,Japanese Restaurant,5.9,2,Sandwich Place,Chinese Restaurant,Grocery Store,Donut Shop,Shoe Store,Japanese Restaurant,Mexican Restaurant,Café,Pizza Place,Bar
28,Lilliana's Restaurant,Cajun / Creole Restaurant,7.8,2,Park,Ice Cream Shop,Shipping Store,Mexican Restaurant,Italian Restaurant,Intersection,Cajun / Creole Restaurant,Pizza Place,Credit Union,Salon / Barbershop
34,Dairyland Family Restaurant,Diner,7.1,2,Bar,Bakery,Gym,Diner,Park,Fast Food Restaurant,Pizza Place,Food,Intersection,Cajun / Creole Restaurant
36,Otto's Restaurant & Bar,American Restaurant,8.3,2,Sandwich Place,Health & Beauty Service,Music Store,Bakery,Salon / Barbershop,Movie Theater,Lingerie Store,Pizza Place,Photography Studio,Pharmacy
41,Jamerica Carribean Restaurant,Caribbean Restaurant,8.1,2,Coffee Shop,Art Gallery,Park,Bakery,Pizza Place,Thrift / Vintage Store,Music Venue,Liquor Store,Mexican Restaurant,Ethiopian Restaurant
42,The Curve Restaurant,American Restaurant,6.2,2,Chinese Restaurant,Sandwich Place,Grocery Store,Japanese Restaurant,Shoe Store,Pizza Place,Mexican Restaurant,Bar,Donut Shop,BBQ Joint


In [87]:
mad_merged2[mad_merged2['Cluster Labels'] == 3]

,name,categories,rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Coliseum Bar and Restaurant,American Restaurant,6.7,3,Park,Steakhouse,American Restaurant,Music Venue,Trail,Hotel,Athletics & Sports,Bar,Donut Shop,Dive Bar


### Combine the nearby venues number into each cluster respectively
The number of nearby venues for each cluster will be combined into each cluster data, we want to see the average number of nearby venues for each venues.

In [89]:
cluster0 = mad_merged2[mad_merged2['Cluster Labels'] == 0]
cluster0 = cluster0[['name','categories','rating','1st Most Common Venue']]
cluster0 = cluster0.join(nearVenues_Num.set_index('Restaurant Name'), on='name')
print('The average rating score of cluster 0 is', cluster0['rating'].mean(), 
      'and the average counts of nearby venues is', cluster0['Nearby Venue counts'].mean())

cluster1 = mad_merged2[mad_merged2['Cluster Labels'] == 1]
cluster1 = cluster1[['name','categories','rating','1st Most Common Venue']]
cluster1 = cluster1.join(nearVenues_Num.set_index('Restaurant Name'), on='name')
print('The average rating score of cluster 1 is', cluster1['rating'].mean(), 
      'and the average counts of nearby venues is', cluster1['Nearby Venue counts'].mean())

cluster2 = mad_merged2[mad_merged2['Cluster Labels'] == 2]
cluster2 = cluster2[['name','categories','rating','1st Most Common Venue']]
cluster2 = cluster2.join(nearVenues_Num.set_index('Restaurant Name'), on='name')
print('The average rating score of cluster 2 is', cluster2['rating'].mean(), 
      'and the average counts of nearby venues is', cluster2['Nearby Venue counts'].mean())

cluster3 = mad_merged2[mad_merged2['Cluster Labels'] == 3]
cluster3 = cluster3[['name','categories','rating','1st Most Common Venue']]
cluster3 = cluster3.join(nearVenues_Num.set_index('Restaurant Name'), on='name')
print('The average rating score of cluster 3 is', cluster3['rating'].mean(), 
      'and the average counts of nearby venues is', cluster3['Nearby Venue counts'].mean())

The average rating score of cluster 0 is 8.0 and the average counts of nearby venues is 46.0
The average rating score of cluster 1 is 7.126666666666667 and the average counts of nearby venues is 74.4
The average rating score of cluster 2 is 7.181818181818182 and the average counts of nearby venues is 20.727272727272727
The average rating score of cluster 3 is 6.7 and the average counts of nearby venues is 10.0


## Results and Discussion

1. Compare ratings
By ranking the restaurant rating scores, we can see the top restaurants are ... 
After averaging the ratings over the categories and then sorting restaurant based on the ratings, we can see the top 3 popular restaurant actegories in Madison area are Gastropub, Indonesian Restaurant and Sushi Restaurant, and Japanese restaurant, Afghan Restaurant and Thai restaurant are not popular according to our data. 

2. Nearby venues
Top 4 restaurants, ranked by the number of nearby venues, are Nick's Restaurant & Bar, Vientiane Palace, The Old Fashioned Tavern & Restaurant and Restaurant Muramoto. We can choose these areas to locate the new restaurants.From the nearby venues, we can learn about the customer/people flow. Usually, the large amount of venues around means high customer flow, and means this restaurant might has good business.  

## Conclusion
In conclusion, I will recommend the restaurant investors to start a restaurant from these three types: Gastropub, Indonesian and Sushi, since these restaurants has more high rating score which means they more popular compare with other type of restarants in Madison area. Starting with either type gives more potential business to the investors, because people are more willing to go their restaurants. While on the other hand, I won't recommend to start Japanese restaurant, Afghan Restaurant or Thai restaurant, beacause the low average rating score show the peole are not tending to go these restaurants.  